In [18]:
audiopth = "곰세마리.wav"
label = "곰 세마리가 한집에 있어 아빠곰 엄마곰 애기곰 아빠곰은 뚱뚱해 엄마 곰은 날"

In [19]:
import sys,os
current_dir = os.getcwd()
sys.path.append(os.path.dirname(os.path.abspath(current_dir)))
from utils.Audio import PrintAudioInfo, GetAudio, remove_Silence
from utils.levenshtein_distance import infer
from transformers import Wav2Vec2ForCTC
from transformers import Wav2Vec2Processor
import torch
import numpy as np

model_id = 'hongseongpil/wav2vec2-vocals'
model = Wav2Vec2ForCTC.from_pretrained(model_id,output_attentions=True)
model.eval()
processor = Wav2Vec2Processor.from_pretrained(model_id)

from utils.Tokenize_Kor import decompose_tokens
audio = remove_Silence(GetAudio(audiopth))[:20*1000]
input = processor(np.array(audio.set_channels(1).get_array_of_samples(),dtype=np.float32), sampling_rate=16000, return_tensors="pt").input_values[0]
with torch.no_grad():
    input_values = torch.tensor(input).unsqueeze(0)
    logits = model(input_values).logits

predlogits = torch.argmax(logits, dim=-1)[0]
outputs = processor.decode(predlogits,output_char_offsets=True)

decomponsed = decompose_tokens(label)
while ' ' in decomponsed[0]:
    index = decomponsed[0].index(' ')
    del decomponsed[0][index]
    del decomponsed[1][index]

origintext = "".join(decomponsed[0])
predtext = outputs['text'].replace(' ','')
time_offset = model.config.inputs_to_logits_ratio / processor.feature_extractor.sampling_rate

Some weights of the model checkpoint at hongseongpil/wav2vec2-vocals were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at hongseongpil/wav2vec2-vocals and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN this mode

In [20]:
print("origintext:", origintext , 'len',len(origintext))
print("predtext:",predtext, 'len',len(predtext))
PrintAudioInfo(audio)

for w in outputs['char_offsets']:
    start_offset = round (w[ "start_offset" ] * time_offset, 2 )
    end_offset = round (w[ "end_offset" ] * time_offset, 2 )
    print('word : ',w['char'],'start_offset :',start_offset,'end_offset :',end_offset)

origintext: ㄱㅗㅁㅅㅔㅁㅏㄹㅣㄱㅏㅎㅏㄴㅈㅣㅂㅔㅇㅣㅆㅓㅇㅏㅃㅏㄱㅗㅁㅇㅓㅁㅁㅏㄱㅗㅁㅇㅐㄱㅣㄱㅗㅁㅇㅏㅃㅏㄱㅗㅁㅡㄴㄸㅜㅇㄸㅜㅇㅎㅐㅇㅓㅁㅁㅏㄱㅗㅁㅡㄴㄴㅏㄹ len 74
predtext: ㄱㅓㅇㅓㄴㅅㅔㅇㅁㅏㅇㅏㄹㅣㅇㅣㄱㅏㅇㅏㅎㅏㅇㅏㅇㅈㅣㅂㅔㅇㅣㅆㅓㅇㅏㅇㅏㅃㅏㄱㅗㅇㅁㅇㅓㅁㅏㄱㅓㅇㅗㅁㅇㅐㅎㄹㅇㅣㄱㅗㅇㅗㅇㅇㅏㅃㅏㅇㅏㄱㅗㅇㅗㅁㅡㅇㅡㄴㄸㅜㅇㅜㅁㅌㅜㅇㅜㅁㅎㅐㅇㅐㅇㅓㅁㅁㅏㅇㅏㄱㅓㅇㅓㅁㅡㄴㄴ len 107
Channels: 2
Sample rate: 16000
Duration:  20.0
Bit depth: 2 bits
len samples: 640000


word :  ㄱ start_offset : 7.26 end_offset : 7.28
word :  ㅓ start_offset : 7.3 end_offset : 7.32
word :  ㅇ start_offset : 7.32 end_offset : 7.36
word :  ㅓ start_offset : 7.42 end_offset : 7.46
word :  ㄴ start_offset : 7.52 end_offset : 7.54
word :  ㅅ start_offset : 7.68 end_offset : 7.7
word :  ㅔ start_offset : 7.74 end_offset : 7.76
word :  ㅇ start_offset : 7.76 end_offset : 7.78
word :  ㅁ start_offset : 7.94 end_offset : 7.96
word :  ㅏ start_offset : 8.0 end_offset : 8.04
word :  ㅇ start_offset : 8.04 end_offset : 8.06
word :  ㅏ start_offset : 8.06 end_offset : 8.12
word :  ㄹ start_offset : 8.18 end_offset : 8.22
word :  ㅣ start_offset : 8.26 end_offset : 8.28
word :  ㅇ start_offset : 8.28 end_offset : 8.32
word :  ㅣ start_offset : 8.48 end_offset : 8.52
word :  ㄱ start_offset : 8.64 end_offset : 8.66
word :  ㅏ start_offset : 8.68 end_offset : 8.7
word :  ㅇ start_offset : 8.7 end_offset : 8.74
word :  ㅏ start_offset : 8.9 end_offset : 8.94
word :  ㅎ start_offset : 9.0 end_offset : 9.04

In [21]:
infered = infer(origintext,predtext)
print("Matching....\n")

for item in infered:
    pred ="".join([predtext[j-1] for j in item[1]])
    print("origintext:", origintext[item[0]-1],"predtext:",pred)

Matching....

origintext: ㄱ predtext: ㄱ
origintext: ㅗ predtext: ㅓㅇㅓ
origintext: ㅁ predtext: ㄴ
origintext: ㅅ predtext: ㅅ
origintext: ㅔ predtext: ㅔㅇ
origintext: ㅁ predtext: ㅁ
origintext: ㅏ predtext: ㅏㅇㅏ
origintext: ㄹ predtext: ㄹ
origintext: ㅣ predtext: ㅣㅇㅣ
origintext: ㄱ predtext: ㄱ
origintext: ㅏ predtext: ㅏㅇㅏ
origintext: ㅎ predtext: ㅎ
origintext: ㅏ predtext: ㅏㅇㅏ
origintext: ㄴ predtext: ㅇ
origintext: ㅈ predtext: ㅈ
origintext: ㅣ predtext: ㅣ
origintext: ㅂ predtext: ㅂ
origintext: ㅔ predtext: ㅔ
origintext: ㅇ predtext: ㅇ
origintext: ㅣ predtext: ㅣ
origintext: ㅆ predtext: ㅆ
origintext: ㅓ predtext: ㅓ
origintext: ㅇ predtext: ㅇㅏㅇ
origintext: ㅏ predtext: ㅏ
origintext: ㅃ predtext: ㅃ
origintext: ㅏ predtext: ㅏ
origintext: ㄱ predtext: ㄱ
origintext: ㅗ predtext: ㅗㅇ
origintext: ㅁ predtext: ㅁ
origintext: ㅇ predtext: ㅇ
origintext: ㅓ predtext: ㅓ
origintext: ㅁ predtext: ㅁ
origintext: ㅁ predtext: ㅁ
origintext: ㅏ predtext: ㅏ
origintext: ㄱ predtext: ㄱ
origintext: ㅗ predtext: ㅓㅇㅗ
origintext: ㅁ predtext: ㅁ
originte

In [22]:
from collections import Counter
count_dict = dict(Counter(decomponsed[1]))
count_list = [[num, count] for num, count in count_dict.items()]
result = []
phoneme_index = 0
for i in count_list:
    start_index = infered[phoneme_index][1][0]-1
    end_index = infered[phoneme_index+i[1]-1][1][-1]-1
    phoneme_index += i[1]
    start_offset = outputs['char_offsets'][start_index]["start_offset"]
    end_offset = outputs['char_offsets'][end_index]["end_offset"]
    result.append({'origin' : label[i[0]],'start':round (start_offset* time_offset, 2 ),'end' :round (end_offset* time_offset, 2 )})
    
result

[{'origin': '곰', 'start': 7.26, 'end': 7.54},
 {'origin': '세', 'start': 7.68, 'end': 7.78},
 {'origin': '마', 'start': 7.94, 'end': 8.12},
 {'origin': '리', 'start': 8.18, 'end': 8.52},
 {'origin': '가', 'start': 8.64, 'end': 8.94},
 {'origin': '한', 'start': 9.0, 'end': 9.36},
 {'origin': '집', 'start': 9.52, 'end': 9.6},
 {'origin': '에', 'start': 9.74, 'end': 9.82},
 {'origin': '있', 'start': 9.82, 'end': 10.14},
 {'origin': '어', 'start': 10.42, 'end': 10.5},
 {'origin': '아', 'start': 10.5, 'end': 10.94},
 {'origin': '빠', 'start': 11.08, 'end': 11.2},
 {'origin': '곰', 'start': 11.34, 'end': 11.6},
 {'origin': '엄', 'start': 11.8, 'end': 12.02},
 {'origin': '마', 'start': 12.0, 'end': 12.12},
 {'origin': '곰', 'start': 12.24, 'end': 12.5},
 {'origin': '애', 'start': 12.72, 'end': 13.04},
 {'origin': '기', 'start': 13.2, 'end': 13.5},
 {'origin': '곰', 'start': 13.62, 'end': 14.1},
 {'origin': '아', 'start': 14.5, 'end': 14.68},
 {'origin': '빠', 'start': 14.96, 'end': 15.3},
 {'origin': '곰', 'start

In [23]:
for w in result:
    start = w['start']
    end = w['end']
    print(w['origin'],w['start'],w['end'])
    PrintAudioInfo(audio[start*1000:end*1000])

곰 7.26 7.54
Channels: 2
Sample rate: 16000
Duration:  0.28
Bit depth: 2 bits
len samples: 8960


세 7.68 7.78
Channels: 2
Sample rate: 16000
Duration:  0.1
Bit depth: 2 bits
len samples: 3200


마 7.94 8.12
Channels: 2
Sample rate: 16000
Duration:  0.1799375
Bit depth: 2 bits
len samples: 5758


리 8.18 8.52
Channels: 2
Sample rate: 16000
Duration:  0.34
Bit depth: 2 bits
len samples: 10880


가 8.64 8.94
Channels: 2
Sample rate: 16000
Duration:  0.3
Bit depth: 2 bits
len samples: 9600


...

엄 18.14 18.6
Channels: 2
Sample rate: 16000
Duration:  0.46
Bit depth: 2 bits
len samples: 14720


마 18.56 18.94
Channels: 2
Sample rate: 16000
Duration:  0.38
Bit depth: 2 bits
len samples: 12160


곰 19.06 19.44
Channels: 2
Sample rate: 16000
Duration:  0.38
Bit depth: 2 bits
len samples: 12160


은 19.52 19.94
Channels: 2
Sample rate: 16000
Duration:  0.42
Bit depth: 2 bits
len samples: 13440


날 19.9 19.94
Channels: 2
Sample rate: 16000
Duration:  0.04
Bit depth: 2 bits
len samples: 1280


In [24]:
print("Matching....\n")

for item in infered:
    pred ="".join([predtext[j-1] for j in item[1]])
    print("origintext:", origintext[item[0]-1],"predtext:",pred)

for w in outputs['char_offsets']:
    start_offset = round (w[ "start_offset" ] * time_offset, 2 )
    end_offset = round (w[ "end_offset" ] * time_offset, 2 )
    print('word : ',w['char'],'start_offset :',start_offset,'end_offset :',end_offset)

    display(audio[start_offset*1000:end_offset*1000])



Matching....

origintext: ㄱ predtext: ㄱ
origintext: ㅗ predtext: ㅓㅇㅓ
origintext: ㅁ predtext: ㄴ
origintext: ㅅ predtext: ㅅ
origintext: ㅔ predtext: ㅔㅇ
origintext: ㅁ predtext: ㅁ
origintext: ㅏ predtext: ㅏㅇㅏ
origintext: ㄹ predtext: ㄹ
origintext: ㅣ predtext: ㅣㅇㅣ
origintext: ㄱ predtext: ㄱ
origintext: ㅏ predtext: ㅏㅇㅏ
origintext: ㅎ predtext: ㅎ
origintext: ㅏ predtext: ㅏㅇㅏ
origintext: ㄴ predtext: ㅇ
origintext: ㅈ predtext: ㅈ
origintext: ㅣ predtext: ㅣ
origintext: ㅂ predtext: ㅂ
origintext: ㅔ predtext: ㅔ
origintext: ㅇ predtext: ㅇ
origintext: ㅣ predtext: ㅣ
origintext: ㅆ predtext: ㅆ
origintext: ㅓ predtext: ㅓ
origintext: ㅇ predtext: ㅇㅏㅇ
origintext: ㅏ predtext: ㅏ
origintext: ㅃ predtext: ㅃ
origintext: ㅏ predtext: ㅏ
origintext: ㄱ predtext: ㄱ
origintext: ㅗ predtext: ㅗㅇ
origintext: ㅁ predtext: ㅁ
origintext: ㅇ predtext: ㅇ
origintext: ㅓ predtext: ㅓ
origintext: ㅁ predtext: ㅁ
origintext: ㅁ predtext: ㅁ
origintext: ㅏ predtext: ㅏ
origintext: ㄱ predtext: ㄱ
origintext: ㅗ predtext: ㅓㅇㅗ
origintext: ㅁ predtext: ㅁ
originte

word :  ㅓ start_offset : 7.3 end_offset : 7.32


word :  ㅇ start_offset : 7.32 end_offset : 7.36


word :  ㅓ start_offset : 7.42 end_offset : 7.46


word :  ㄴ start_offset : 7.52 end_offset : 7.54


...

word :  ㅓ start_offset : 19.38 end_offset : 19.44


word :  ㅁ start_offset : 19.52 end_offset : 19.54


word :  ㅡ start_offset : 19.62 end_offset : 19.7


word :  ㄴ start_offset : 19.76 end_offset : 19.8


word :  ㄴ start_offset : 19.9 end_offset : 19.94
